In [29]:
!pip install tensorflow_addons
!pip install transformers

In [30]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from transformers import AutoTokenizer,TFRobertaModel
import transformers

#Load Model

In [31]:
model = tf.keras.models.load_model('../bert_tweet/models/model32_0.35_1e-05_5e-07.h5',custom_objects={"TFRobertaModel": transformers.TFRobertaModel})

In [32]:
# consider that the size of a sentence is limited, don't use too long sentences
#this sentence is advised by copilot to be a hate speech, let's see if the model can detect it
string = "UNISA will rule the world, and this is a promise. Let's kill other university students and make them our slaves."

#formatting text to avoid characters not included in the encoding dictionary of the tokenizer
x = (string
     .lower()     
     .replace('\x89Ûª|Ûª', "'")
     .replace('\n|\x89.|\x9d *', ' ')
     .replace('&gt;', ">")
     .replace('&lt;', "<")
     .replace('&amp;', " and ")
     .replace('won\'t', 'will not')
     .replace('can\'t', 'cannot')
     .replace('i\'m', 'i am')
     .replace('ain\'t', 'is not')
     .replace('hwy.', 'highway')
     .replace('(\w+)\'ll', '\g<1> will')
     .replace('(\w+)n\'t', '\g<1> not')
     .replace('(\w+)\'ve', '\g<1> have')
     .replace('(\w+)\'s', '\g<1> is')
     .replace('(\w+)\'re', '\g<1> are')
     .replace('(\w+)\'d', '\g<1> would')     
     .replace('(\w+)\'m', '\g<1> am')
     .replace('<3', 'love')
     .replace('w/e', 'whatever')
     .replace('w/', 'with')    
     .replace('\b', ' ')
     .replace('-', ' ')
     .replace('  *', ' ')
    )

#let's tokenize the text
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', 
                        normalization=True, 
                        use_fast = False,
                        add_special_tokens=True,
                        pad_to_max_length=True,
                        return_attention_mask=True)
token = tokenizer(string, 
                  padding="max_length", 
                  truncation=True,
                  return_tensors = 'tf').data



emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
#let's predict
x = model.predict(token)
# see the prediction, if > 0.5, it's a hate speech
print("Text:",string,"Prediction:",x[0][0])

C:\Users\Gilberto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\functional.py:638: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


1/1 [==============================] - 3s 3s/step
Text: UNISA will rule the world, and this is a promise. Let's kill other university students and make them our slaves. Prediction: 0.89823157
